In [5]:
# Import libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import get_peft_model, LoraConfig


In [8]:
# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load and prepare the dataset
dataset = load_dataset("raeidsaqur/nifty")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [11]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# tokenized_datasets = dataset.map(preprocess_function, batched=True)
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# tokenized_datasets.set_format("torch")

# Load the pre-trained FinGPT model

In [12]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Wrap the model with LoRA
lora_model = LoraModel(model)

# Training parameters
training_args = TrainingArguments(
    output_dir="./finetuned_fingpt",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=1000,
    load_best_model_at_end=True
)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize the Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
)

NameError: name 'LoraModel' is not defined

In [2]:
# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the fine-tuned model
trainer.save_model("./finetuned_fingpt")

In [2]:
# Load the fine-tuned model for inference
model = GPT2LMHeadModel.from_pretrained("./finetuned_fingpt")
tokenizer = GPT2Tokenizer.from_pretrained("./finetuned_fingpt")

# Inference pipeline
from transformers import pipeline
nlp_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate financial insights
text = "The market trend for the upcoming quarter is"
generated_text = nlp_pipeline(text, max_length=100, num_return_sequences=1)
print(generated_text)

ImportError: cannot import name 'LoraModel' from 'lora' (/home/jovyan/FinSightAI-Shashank/Virtual/myenv/lib/python3.10/site-packages/lora/__init__.py)